# Data Engineering Zoom Camp 5 - Batch processing

## Spark session setup

First of all, we need to setup the system.
I will assume that the system already have Python and Java 17 installed.
The setup commands can be executed just by running the `setup.sh` script.
This will install Spark and download the required data files.

Once the setup have been done, you can start tunning this notebook.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("homework").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/02/23 17:07:13 WARN Utils: Your hostname, Baby resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/02/23 17:07:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/23 17:07:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## First question

In [2]:
spark.version

'3.5.4'

## Second question

In [3]:
yellow_df = spark.read.parquet("yellow_tripdata_2024-10.parquet")
yellow_df.repartition(4).write.parquet("yellow_tripdata_2024-10-repartitioned.parquet")

In [4]:
!ls -lh yellow_tripdata_2024-10-repartitioned.parquet

total 90M
-rw-r--r-- 1 jcamps jcamps   0 Feb 23 17:07 _SUCCESS
-rw-r--r-- 1 jcamps jcamps 23M Feb 23 17:07 part-00000-6724562b-525e-4cdf-af40-a96dea447378-c000.snappy.parquet
-rw-r--r-- 1 jcamps jcamps 23M Feb 23 17:07 part-00001-6724562b-525e-4cdf-af40-a96dea447378-c000.snappy.parquet
-rw-r--r-- 1 jcamps jcamps 23M Feb 23 17:07 part-00002-6724562b-525e-4cdf-af40-a96dea447378-c000.snappy.parquet
-rw-r--r-- 1 jcamps jcamps 23M Feb 23 17:07 part-00003-6724562b-525e-4cdf-af40-a96dea447378-c000.snappy.parquet


## Third question

In [5]:
from pyspark.sql import functions as F

yellow_df.withColumn("date", F.to_date("tpep_pickup_datetime", "d-M-y")).filter(
    (F.col("date") == F.lit("2024-10-15"))
).count()

125567

## Fourth question

In [6]:
yellow_df.withColumn(
    "duration", F.col("tpep_dropoff_datetime") - F.col("tpep_pickup_datetime")
).agg(F.max("duration")).collect()[0][0].total_seconds() / 3600

162.61777777777777

## Fifth question

This is a theory question. The port for the Spark dashboard is the 4040, as stated in the [documentation](https://spark.apache.org/docs/latest/configuration.html#spark-ui).

## Sixth question

In [7]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType

schema = StructType(
    [
        StructField("LocationID", IntegerType()),
        StructField("Borough", StringType()),
        StructField("Zone", StringType()),
        StructField("service_zone", StringType()),
    ]
)
zone_df = spark.read.schema(schema).option("header", "true").csv("taxi_zone_lookup.csv")
count_trips_df = yellow_df.groupBy("PULocationID").count().orderBy("count")
minimum_trips = count_trips_df.first()["count"]
count_trips_df.filter(F.col("count") == minimum_trips).join(
    zone_df, F.col("PULocationID") == F.col("LocationID")
).select("Zone").show()

+--------------------+
|                Zone|
+--------------------+
|Governor's Island...|
+--------------------+

